In [27]:
import random
import numpy as np

# graphs
import matplotlib.pyplot as plt
import networkx as nx
from networkx.generators.random_graphs import erdos_renyi_graph, complete_graph
from networkx.generators import circulant_graph

In [59]:
class DAPDG:
    
    def __init__(self, F, grad_F, eta=0.3, alpha=0.45, beta=0.01, theta=0.78, sigma=0.12, eps=1e-16, verbose=False):
        """
        F: function
        grad_F: function - F'(x)
        L: float - smoothness constante, L > mu
        mu: float - strong convexity constant
        sigma: float - ???
        eps: float - threshold to stop iterations
        """
        assert 0 < sigma <= 1
        assert 0 < eps
        
        self.eta = eta
        self.alpha = alpha
        self.beta = beta
        self.theta = theta
        self.sigma = sigma
        
        self.eps = eps        
        self.F = F
        self.grad_F = grad_F

        self.verbose = verbose
        
        
    def init_x0(self, B, W, N):
        M = W.shape[0]
        N = B.shape[1]
        A = np.vstack([np.kron(np.ones((M, M)), B), gamma * np.kron(W, np.ones((N, N)))])
        return (A * np.random.rand(N * M)).reshape(M, N)


    def opt_step(self, x, x_f, A):
        x_g = self.theta * x + (1 - self.theta) * x_f
        
        grad_F_x_g = np.array([grad_F[i](x_g[i]) for i in range(M)])
        x_new = x + self.eta * self.alpha * (x_g - x) - \
            self.eta * self.beta * (A.T.dot(A.dot(x.reshape(N*M)))).reshape(M,N) - \
            self.eta * grad_F_x_g
        
        x_f = x_g + self.sigma * (x_new - x)
        
        return x_new, x_f


    def optimize(self, x, B, W, gamma=1, steps_count=100):
        M = W.shape[0]
        N = B.shape[1]
        A = np.vstack([np.kron(np.ones((M, M)), B), gamma * np.kron(W, np.ones((N, N)))])
        
        x_f = x
        for i in range(steps_count):
            if sum(sum(grad_F[j](x[j]) ** 2 for j in range(M))) < self.eps:
                return i, x
            
            x, x_f = self.opt_step(x, x_f, A)
            if self.verbose and i % self.verbose == 0:
                print(f'Step {i}: {x}')

        if self.verbose:
            print('Not converged')
        return False, x

$$f_i(x) = \frac{1}{2}\| C_i x - d_i \|_2^2 + \frac{\theta}{2} \| x \|_2^2 $$

In [60]:
N = 5  # dimension of space 
M = 3  # count of nodes
P = 4  # dimension of constraints

In [61]:
C = np.random.rand(M, N)
d = np.random.rand(M, N)
theta = 0.5

def create_func(C_i, d_i, theta):
    def f(x):
        first_part = C_i * x - d_i
        first_part = 0.5 * np.sum(first_part ** 2)
        second_part = theta / 2 * np.sum(x ** 2)
        return first_part + second_part
        
    def grad_f(x):
        return C_i ** 2 * x - d_i / 2 + theta * x
    
    return f, grad_f
    
F = [create_func(C[i], d[i], theta)[0] for i in range(M)]
grad_F = [create_func(C[i], d[i], theta)[1] for i in range(M)]

In [62]:
N = 5
M = 3
P = 4
x = np.random.rand(M, N)
B = np.random.rand(P, N)
B[-1] = B[:-1].sum(axis=0)  # to create non-trivial kernel
G = circulant_graph(M, [1])
W = nx.laplacian_matrix(G).A  # may be better not to convert into numpy array

In [63]:
opt = DAPDG(F, grad_F, eps=1e-16)
opt.optimize(x, B, W)

(False,
 array([[-0.15952721,  0.0996692 ,  0.44632422, -0.03562857,  0.04354121],
        [-0.01535027, -0.1203439 ,  0.37038599,  0.30530841,  0.17242742],
        [-0.10111642,  0.10444119, -0.17754932,  0.33236817,  0.40637627]]))

In [64]:
import itertools
from tqdm import tqdm

min_i = 1000
best_success = -1
success = []
for eta, alpha, beta, theta, sigma in tqdm(itertools.product(np.linspace(0.01, 1, 10), np.linspace(0.01, 1, 10), np.linspace(0.01, 1, 10),
                                                             np.linspace(0.01, 1, 10), np.linspace(0.01, 1, 10))):
    x = np.random.rand(N * M)
    opt = DAPDG(F, grad_F, eta=eta, alpha=alpha, beta=beta, theta=theta, sigma=sigma, eps=1e-16)
    res = opt.optimize(x, B, W)
    if res[0]:
        success.append((res, eta, alpha, beta, theta, sigma))
        if res[0] < min_i:
            min_i = res[0]
            best_success = len(success) - 1

0it [00:00, ?it/s]


ValueError: operands could not be broadcast together with shapes (15,) (3,5) 

In [65]:
success[1991]

IndexError: list index out of range